# WikiRecentPhase3
Buil
Shred the page, discover if it had and image and render the image. 

## Streams processing + windowing + 

In [1]:
# Install components
!pip install sseclient
!pip install --user --upgrade streamsx

You are using pip version 18.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Requirement already up-to-date: streamsx in /Users/siegenth/.local/lib/python3.5/site-packages (1.12.9)
You are using pip version 18.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
# Setup 
import pandas as pd

from IPython.core.debugger import set_trace
from IPython.display import display, clear_output

from statistics import mean
from collections import deque
from collections import Counter

import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import Button, HBox, VBox, Layout

from bs4 import BeautifulSoup

%matplotlib inline

from sseclient import SSEClient as EventSource

from ipywidgets import Button, HBox, VBox, Layout

from  functools import lru_cache
import requests

from streamsx.topology.topology import *
import streamsx.rest as rest
from streamsx.topology import context

## Support functions for Jupyter

In [3]:
def catchInterrupt(func):
    """decorator : when interupt occurs the display is lost if you don't catch it
       TODO * <view>.stop_data_fetch()  # stop
       
    """
    def catch_interrupt(*args, **kwargs):
        try: 
            func(*args, **kwargs)
        except (KeyboardInterrupt): pass
    return catch_interrupt

#
# Support for locating/rendering views.
def display_view_stop(eventView, period=2):
    """Wrapper for streamsx.rest_primitives.View.display() to have button. """
    button =  widgets.Button(description="Stop Updating")
    display(button)
    eventView.display(period=period) 
    def on_button_clicked(b):
        eventView.stop_data_fetch()
        b.description = "Stopped"
    button.on_click(on_button_clicked)

def view_events(views):
    """
    Build interface to display a list of views and 
    display view when selected from list.
     
    """
    view_names = [view.name for view in views]
    nameView = dict(zip(view_names, views))    
    select = widgets.RadioButtons(
        options = view_names,
        value = None,
        description = 'Select view to display',
        disabled = False
    )
    def on_change(b):
        if (b['name'] == 'label'):
            clear_output(wait=True)
            [view.stop_data_fetch() for view in views ]
            display(select)
            display_view_stop(nameView[b['new']], period=2)
    select.observe(on_change)
    display(select)

def find_job(instance, job_name=None):
    """locate job within instance"""
    for job in instance.get_jobs():    
        if job.applicationName.split("::")[-1] == job_name:
            return job
    else:
        return None

def display_views(instance, job_name):
    "Locate/promote and display all views of a job"
    job = find_job(instance, job_name=job_name)
    if job is None:
        print("Failed to locate job")
    else:
        views = job.get_views()
        view_events(views)

def list_jobs(_instance=None, cancel=False):
    """
    Interactive selection of jobs to cancel.
    
    Prompts with SelectMultiple widget, if thier are no jobs, your presente with a blank list.
    
    """
    active_jobs = { "{}:{}".format(job.name, job.health):job for job in _instance.get_jobs()}

    selectMultiple_jobs = widgets.SelectMultiple(
        options=active_jobs.keys(),
        value=[],
        rows=len(active_jobs),
        description = "Cancel jobs(s)" if cancel else "Active job(s):",
        layout=Layout(width='60%')
    )
    cancel_jobs = widgets.ToggleButton(
        value=False,
        description='Cancel',
        disabled=False,
        button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Delete selected jobs',
        icon="stop"
    )
    def on_value_change(change):
        for job in selectMultiple_jobs.value:
            print("canceling job:", job, active_jobs[job].cancel())
        cancel_jobs.disabled = True
        selectMultiple_jobs.disabled = True

    cancel_jobs.observe(on_value_change, names='value')
    if cancel:
        return HBox([selectMultiple_jobs, cancel_jobs])
    else:
        return HBox([selectMultiple_jobs])

# Connect to the server :  ICP4D or Cloud instance. 
Attempt to import if fails the cfg will not be defined we know were using 
Cloud.

In [4]:
# ICP4D  - injected by 'Connected to instance' menu item
try:
    from icpd_core import icpd_util
    cfg = icpd_util.get_service_instance_details(name='sample-icp1')
    cfg[context.ConfigParams.SSL_VERIFY] = False
    instance = rest.Instance.of_service(cfg)
    print("Within ICP4D")
except ImportError:
    cfg = None
    print("Outside ICP4D")

Outside ICP4D


In [5]:
# disable 'InsecureRequestWarning'  - must be put after startup
if cfg is not None:
    import urllib3
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [6]:
# NOT ICP4D : cloud access - 
if cfg is None:
    import credential   # remove @ ICP4D
    import common
    # TODO * check if instance is up
    # - link up to first cell (can you do a test and execute)
    sc = rest.StreamingAnalyticsConnection(service_name='Streaming3Turbine', vcap_services={'streaming-analytics':[{'name':'Streaming3Turbine','credentials':credential.streaming3Turbine}]})
    instance = sc.get_instances()[0]
    # Render the views.....

In [7]:
list_jobs(instance)

HBox(children=(SelectMultiple(description='Active job(s):', layout=Layout(width='60%'), options=('ipythoninput56f6b242d49e00::WikiPhase3_4:healthy',), rows=1, value=()),))

# Foundation : Code from WikiRecentPhase2
This page is a is build upon the WikieRecentPhase2, the contents are included refer to the originating code for
details.

In [8]:
def get_events():
    """fetch recent changes from wikievents site using SSE"""
    for change in EventSource('https://stream.wikimedia.org/v2/stream/recentchange'):
        if len(change.data):
            try:
                obj = json.loads(change.data)
            except json.JSONDecodeError as err:
                print("JSON l1 error:", err, "Invalid JSON:", change.data)
            except json.decoder.JSONDecodeError as err:
                print("JSON l2 error:", err, "Invalid JSON:", change.data)
            else:
                yield(obj)


In [9]:
class sum_aggregate():
    def __init__(self, sum_map={'new_len':'newSum','old_len':'oldSum','delta_len':'deltaSum' }):
        """
        Summation of column(s) over a window's list of objects. 
        On a windows list objects, perform summation on the summary_field, output to result_field
        """
        self.sum_map = sum_map
    def __call__(self, tuples):    
        summaries = dict()
        for summary_field,result_field in self.sum_map.items():
            summation = sum([ele[summary_field] for ele in tuples])
            summaries.update({result_field : summation})
        return(summaries)


In [10]:
import collections
class tally_fields(object):
    def __init__(self, top_count=3, fields=['user', 'wiki', 'title']):
        """
        Tally fields of a list of tuples, output the name and . 
        """
        self.fields = fields
        self.top_count = top_count
    def __call__(self, tuples)->list:
        tallies = dict()
        for field in self.fields:
            stage = [tuple[field] for tuple in tuples if tuple[field] is not None]
            tallies[field] = collections.Counter(stage).most_common(self.top_count)
        return tallies

In [11]:
import csv
class wiki_lang():
    """
    Augment the tuple to include language a description of the wiki the 
    event is destined for. The entry is augmented with language, native and code. 
    
    Mapping is loaded at build time and utilized at runtime.
    """

    def __init__(self, fname="wikimap.csv"):
        self.wiki_map = dict()
        with open(fname, mode='r') as csv_file:
            csv_reader = csv.DictReader(csv_file)
            for row in csv_reader:
                self.wiki_map[row['dbname']] = row

    def __call__(self, tuple):
        if tuple['wiki'] in self.wiki_map:
            key = tuple['wiki']
            tuple['code'] = self.wiki_map[key]['code']
            tuple['language'] = self.wiki_map[key]['in_english']
            tuple['native'] = self.wiki_map[key]['name_language']
        else:
            tuple['code'] = tuple['language'] = tuple['native'] = None
        return tuple


## New functionality...
- find possible link to image
- build and shred the link to find the image. 
- shredding can go down mulitple levels.
- turn on lru_cache() after you determine why it fails

If the title has a URL it's with an image extenstion (.jpg, .gif) it's good bet that and image is being added to an article. The url maybe a a page with the image url embeded within it. Look at using the 'beatifulsoup' package to locate the image within the linked page. With the image you can do image analysis use beautifulsoup. 

In [12]:
#@lru_cache(maxsize=None)
def shred_item_image(url):
    """Shed the item page, seeking image. 
    
    Discover if referencing image by shredding referening url. If it is, dig deeper 
    and extract the 'src' link. 
    
    Locate the image within the page, locate <a class='image' src=**url** ,..>
    
    This traverses two files, pulls the thumbnail ref and follows to fullsize.
    
    Args:
        url: item page to analyse
    
    Returns: 
        If image found [{name,title,org_url},...]
    
    .. warning:: this fetches from wikipedia, requesting too frequenty is bad manners. Uses the lru_cache()
    so it minimises the requests.
    
    This can pick up multiple titles, on a page that is extract, dropping to only one. 
    """
    img_urls = list()
    try:
        rThumb = requests.get(url = url)
        #print(r.content)
        soupThumb = BeautifulSoup(rThumb.content, "html.parser")
        divThumb = soupThumb.find("div", class_="thumb")
        if divThumb is None:
            print("No thumb found", url  )
            return img_urls
        thumbA = divThumb.find("a", class_="image")
        thumbHref = thumbA.attrs['href']

        rFullImage = requests.get(url=thumbHref)
        soupFull = BeautifulSoup(rFullImage.content, "html.parser")
    except Exception as e:
        print("Error request.get, url: {} except:{}".format(url, str(e)))
    else:
        divFull = soupFull.find("div", class_="fullImageLink", id="file")
        if (divFull is not None):
            fullA = divFull.find("a")
            img_urls.append({"title":soupThumb.title.getText(),"img": fullA.attrs['href'],"org_url":url})
    finally:
        return img_urls

In [13]:
#@lru_cache(maxsize=None)
def shred_jpg_image(url):
    """Shed the jpg page, seeking image, the reference begins with 'Fred:' and 
    ends with '.jpg'.
    
    Discover if referencing image by shredding referening url. If it is, dig deeper 
    and extract the 'src' link. 
    
    Locate the image within the page, 
            locate : <div class='fullImageLinks'..>
                         <a href="..url to image" ...>.</a>
                         :
                     </div>  
    Args:
        url: item page to analyse
    
    Returns: 
        If image found [{name,title,org_url='requesting url'},...]
    
    .. warning:: this fetches from wikipedia, requesting too frequenty is bad manners. Uses the lru_cache()
    so it minimises the requests.
    
    """
    img_urls = list()
    try:
        r = requests.get(url = url)
        soup = BeautifulSoup(r.content, "html.parser")
    except Exception as e:
        print("Error request.get, url: {} except:{}".format(url, str(e)))
    else:
        div = soup.find("div", class_="fullImageLink")
        if (div is not None):
            imgA = div.find("a")
            img_urls.append({"title":soup.title.getText(),"img":"https:" + imgA.attrs['href'],"org_url":url})
        else: 
            print("failed to find div for",url)
    finally:
        return img_urls

In [14]:
class soup_image():
    """If the the field_name has a potential for a image we
    
    Return: 
        None : field did not have potenital for an image.
        [] : had potential but no url found. 
        [{title,img,href}]
    """
    def __init__(self, field_name="title", url_base="https://www.wikidata.org/wiki/"):
        self.url_base = url_base
        self.field_name = field_name
    
    def __call__(self, _tuple):
        title = _tuple[self.field_name]
        img_desc = None 
        if (title[0] == "Q"):
            lnk = self.url_base + title
            img_desc = shred_item_image(lnk)
        elif title.startswith("File:") and (title.endswith('.JPG') or title.endswith('.jpg')):
            lnk = self.url_base + title.replace(' ','_')
            img_desc = shred_jpg_image(lnk)
        _tuple['img_desc'] = img_desc
        return _tuple

## Compose and submit

In [2]:
list_jobs(instance, cancel=True)

NameError: name 'list_jobs' is not defined

In [30]:
## If WikiPhase3 is running you may not want to submit, cancel first.

def WikiPhase3(jobName=None, wiki_lang_fname=None):
    """
    Compose topology. 
    -- wiki_lang : csv file mapping database name to langauge

    """
    topo = Topology(name=jobName)
    ### make sure we sseclient in Streams environment.
    topo.add_pip_package('sseclient')
    topo.add_pip_package('bs4')
    
    ## wiki events
    source = topo.source(get_events, name="WikiFeed")
    ## select events generated by humans
    edits = source.filter(lambda x: x['type']=='edit' and x['bot'] is False, name='humanEdits')
    # pare down the humans set of columns
    paredEdits = edits.map(lambda x : {'timestamp':x['timestamp'],'new_len':x['length']['new'],'old_len':x['length']['old'], 'delta_len':x['length']['new'] - x['length']['old'],'wiki':x['wiki'],'user':x['user'],'title':x['title']}, 
                        name="selectMain")
    paredHumanEdits = paredEdits.view(buffer_time=1.0, sample_size=200, name="paredHumanEdits", description="Edits done by humans")

    ## Define window(count)& aggregate
    winAgg = paredEdits.last(100).trigger(20)
    sumAggregate_ = winAgg.aggregate(sum_aggregate(sum_map={'new_len':'newSum','old_len':'oldSum','delta_len':'deltaSum' }), name="sumAggregate")
    aggEdits = sumAggregate_.view(buffer_time=1.0, sample_size=200, name="aggEdits", description="Aggregations of human edits")

    ## Define window(count) & tally edits
    tallyWin = paredEdits.last(100).trigger(10)
    tallyTop = tallyWin.aggregate(tally_fields(fields=['user', 'title'], top_count=10), name="talliesTop")
    talliesCount = tallyTop.view(buffer_time=1.0, sample_size=200, name="talliesCount", description="Top count tallies: user,titles")

    ## augment filterd/pared edits with language

    langAugment = paredEdits.map(wiki_lang(fname=wiki_lang_fname), name="langAugment")
    langAugment.view(buffer_time=1.0, sample_size=200, name="langAugment", description="Language derived from wiki")
    
    ## attempt to extract image using beautifulsoup add img_desc[{}] field
    soupImage = langAugment.map(soup_image(field_name="title", url_base="https://www.wikidata.org/wiki/"),name="imgSoup")
    soupActive = soupImage.filter(lambda x: x['img_desc'] is not None and len(x['img_desc']) > 0, name="soupActive")
    soupActive.view(buffer_time=1.0, sample_size=200, name="soupActive", description="Url souped")

    ## Define window(time) & tally language
    timeLangWin = langAugment.last(datetime.timedelta(minutes=2)).trigger(5)
    timeLanguage = timeLangWin.aggregate(tally_fields(fields=['language'], top_count=10), name="timeLang")
    talliesTime = timeLanguage.view(buffer_time=1.0, sample_size=200, name="talliesTime", description="Top timed tallies: language")
   
    timeLanguage.print(name="printMinLang")
    return ({"topo":topo,"view":{ }})


## ICP4D submission
from streamsx.topology import context
from streamsx.topology.topology import Topology

#cfg=icpd_util.get_service_instance_details(name='sample-icp1')
if cfg is not None:
    resp = WikiPhase3(jobName="WikiPhase3", wiki_lang_fname=os.environ['DSX_PROJECT_DIR']+'/datasets/wikimap.csv')
    # Disable SSL certificate verification if necessary
    cfg[context.ConfigParams.SSL_VERIFY] = False

    submission_result = context.submit("DISTRIBUTED",
                                   resp['topo'], 
                                   config=cfg)
    # The submission_result object contains information about the running application, or job
    if submission_result.job:
        print("JobId: ", submission_result['id'] , "Name: ", submission_result['name'])
## Cloud submission
if cfg is None:
    resp = WikiPhase3(jobName="WikiPhase3", wiki_lang_fname="wikimap.csv")
    submitStatus = common.submitProcess(topology=resp['topo'],
                                        streamsService="Streaming3Turbine",
                                        buildType="DISTRIBUTED",
                                        serviceType="STREAMING_ANALYTIC",
                                        jobName="WikiPhase3",
                                        cancel=True)


Submission parameters:
  - serviceType:STREAMING_ANALYTIC
  - buildType:DISTRIBUTED
  - jobName:WikiPhase3
  - cancel:True
*** No jobs canceled


IntProgress(value=0, bar_style='info', description='Initializing', max=10, style=ProgressStyle(description_width='initial'))

## Render the feed of images two paths.

In [16]:
# Notebook support

def render_image(image_url=None, output_region=None):
    """Write the image into a output region.
    
    Args::
        url: image
        output_region: output region
        
    .. note:: The creation of the output 'stage', if this is not done the image is rendered in the page and
        the output region. 
        
    """
    
    try:
        response = requests.get(image_url)
        stage = widgets.Output(layout={'border': '1px solid green'})
    except:
        print("Error on request : ", image_url)
    else:
        if response.status_code == 200:
            with output_region:
                stage.append_display_data(widgets.Image(
                    value=response.content,
                    #format='jpg',
                    width=300,
                    height=400,
                ))
            output_region.clear_output(wait=True) 

### Render the images - beatifulsoup @ browser.

Tapping the view before the beatifulsoup is invoked : langAugment

Why is this here? This code was created/debugged locally using the WikieRecentPhase3,
when I was done I added the lines to the application to pull them up into the server 
use them. 

```python
  
    soupImage = langAugment.map(soup_image(field_name="title", 
        url_base="https://www.wikidata.org/wiki/"),name="imgSoup")
    soupImage.view(buffer_time=1.0, sample_size=200, name="soupUrl", description="Url souped")

```

Mention....hmm?


In the view from Streams
- extract the item url
- get item page, extract image tag
- render image

The output region is below the cell,

In [ ]:
# Notebook support
status_widget = widgets.Label(value="Status", layout={'border': '1px solid green','width':'30%'})
url_widget = widgets.Label(value="Img URL", layout={'border': '1px solid green','width':'100%'})
image_widget = widgets.Output(layout={'border': '1px solid red','width':'30%','height':'270pt'})
title_widget = widgets.Label(value="Title", layout={'border': '1px solid green','width':'30%'})

dashboard = widgets.VBox([status_widget, image_widget, title_widget, url_widget])
display(dashboard)

In [17]:
# Notebook support
ana_stage = list()
def display_image(tup, image_region=None, title_region=None, url_region=None):
    if tup['img_desc'] is not None and len(tup['img_desc']) > 0:
        display_desc = tup['img_desc'][0]
        ana_stage.append(display_desc)
        title_region.value = "Img Title:{}".format(display_desc['title'] )
        url_region.value = "{}".format(display_desc['img'])
        render_image(image_url=display_desc['img'], output_region=image_region)

In [ ]:
# Notebook support
# setup 
_view = instance.get_views(name="langAugment")[0]
_view.start_data_fetch()
@catchInterrupt
def browser_soup():
    while True:
        view_tuples = _view.fetch_tuples(max_tuples=100, timeout=2)
        for tup in view_tuples:
            status_widget.value = tup['title']
            soup_tuple = soup.__call__(tup)
            display_image(soup_tuple, image_region=image_widget, title_region=title_widget, url_region=url_widget)
    
browser_soup()

In [ ]:
list_jobs(instance, cancel=True)

### Render the images - beatifulsoup @ server

Tapping the after the beatifulsoup is invoked : soupUrl

In [18]:
display_views(instance, job_name="WikiPhase3")

RadioButtons(description='Select view to display', index=3, options=('aggEdits', 'langAugment', 'paredHumanEdits', 'soupActive', 'talliesCount', 'talliesTime'), value=None)

Button(description='Stop Updating', style=ButtonStyle())

VBox(children=(HBox(children=(Text(value='Url souped', description='soupActive', disabled=True), Valid(value=True, description='Fetching', readout='Stopped'))), Output(layout=Layout(border='1px solid black'))))

VBox(children=(Label(value='Status', layout=Layout(border='1px solid green', width='30%')), Output(layout=Layout(border='1px solid red', height='270pt', width='30%')), Label(value='Title', layout=Layout(border='1px solid green', width='30%')), Label(value='Img URL', layout=Layout(border='1px solid green', width='100%'))))

In [1]:
# Notebook support
# setup 
_view = instance.get_views(name="soupActive")[0]
_view.start_data_fetch()

@catchInterrupt
def server_soup():
    while True:
        view_tuples = _view.fetch_tuples(max_tuples=100, timeout=2)
        for soup_tuple in view_tuples:
            status_widget.value = soup_tuple['title']
            display_image(soup_tuple, image_region=image_widget, title_region=title_widget, url_region=url_widget)

server_soup()


NameError: name 'instance' is not defined